In [2]:
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
# read local csv files and put into data frames
data_url = '17zpallagi.csv'
zpallagi_csv = pd.read_csv(data_url)
raw_tax_df = pd.DataFrame(zpallagi_csv)

mhi_data_url = 'MarketHealthIndex_Zip.csv'
market_health_csv = pd.read_csv(mhi_data_url, on_bad_lines='skip', encoding = "ISO-8859-1")
market_health_df = pd.DataFrame(market_health_csv)

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,1,AL,0,1,802640.0,474470.0,99850.0,216600.0,717050.0,44090.0,...,0.0,0.0,64680.0,53602.0,700940.0,1803125.0,698100.0,1796343.0,2860.0,4917.0
1,1,AL,0,2,499070.0,218590.0,137460.0,129760.0,448190.0,26230.0,...,0.0,0.0,77660.0,118725.0,419640.0,1175607.0,416180.0,1165352.0,4250.0,8894.0
2,1,AL,0,3,268590.0,89780.0,134440.0,38280.0,241060.0,14160.0,...,0.0,0.0,67820.0,156752.0,201030.0,560461.0,197060.0,547812.0,5440.0,13482.0
3,1,AL,0,4,170880.0,32180.0,124070.0,11660.0,154120.0,7980.0,...,0.0,0.0,48440.0,141721.0,121930.0,396526.0,118460.0,383588.0,3160.0,12369.0
4,1,AL,0,5,229870.0,22810.0,196990.0,5540.0,208380.0,11120.0,...,50.0,19.0,91100.0,465160.0,138250.0,588068.0,130970.0,529001.0,8430.0,55564.0


,RegionType,RegionName,City,State,Metro,CBSATitle,SizeRank,MarketHealthIndex,SellForGain,PrevForeclosed,ForeclosureRatio,ZHVI,MoM,YoY,ForecastYoYPctChange,StockOfREOs,NegativeEquity,Delinquency,DaysOnMarket
0,Zip,1001,Agawam,MA,"Springfield, MA","""Springfield, MA""",NaN,2.094542,75.00,NaN,5.7575,198700.0,0.353535,5.187930,0.028415,NaN,0.112505,0.072131,77.5
1,Zip,1002,Amherst,MA,"Springfield, MA","""Springfield, MA""",NaN,3.099581,92.31,NaN,NaN,315600.0,-0.221309,3.441495,0.022503,NaN,0.060052,0.050725,93.5
2,Zip,1005,Town of Barre,MA,"Worcester, MA","""Worcester, MA-CT""",NaN,1.054014,50.00,NaN,73.3217,219500.0,0.319927,2.618046,0.039308,NaN,0.079545,0.174603,76.0
3,Zip,1007,Town of Belchertown,MA,"Springfield, MA","""Springfield, MA""",NaN,2.946270,100.00,NaN,NaN,260600.0,-0.572301,0.385208,0.024371,NaN,0.057422,0.034014,88.0
4,Zip,1008,Blandford,MA,"Springfield, MA","""Springfield, MA""",NaN,1.179644,0.00,NaN,NaN,224900.0,0.491510,4.216867,0.025932,NaN,0.071429,0.166667,78.0


In [44]:
# Columns used for both sets
market_health_cols = ['RegionName','MarketHealthIndex','SellForGain','ForeclosureRatio','NegativeEquity','Delinquency',
                      'DaysOnMarket','ZHVI']

zip_tax_cols = ['zipcode','MARS1','MARS2','MARS4','NUMDEP','A00100','N02650', 'A02650','A00200','A00300', 'A00900',
                'A01000','A01700','SCHF','A02300','A02500','N26270','N03220','A03300','N03150','N03210','N03230',
                'A17000','A18425','A18500','A19300','N19570','A19700','A20950','N04800','A04800','A07300','N07180',
                'A07180','N07220','A07220','N09400','A09400','A10600','N11070','A11070','N06500','A06500','A85300']

# filtered and merged together raw data sets 
tax_zip_df = raw_tax_df.filter(zip_tax_cols)
mh_df = market_health_df.filter(market_health_cols, axis=1)
merged_df = pd.merge(tax_zip_df, mh_df, left_on="zipcode", right_on="RegionName")

# meaned out rows with same zip code and dropped the duplicate RegionName / zip field
grouped_df = merged_df.groupby('zipcode').mean().reset_index()
grouped_df = grouped_df.drop(columns=['RegionName'])
# most simpliest data frame
display(grouped_df)


,zipcode,MARS2,MARS4,NUMDEP,A00100,N02650,A02650,A00200,A00300,A00900,...,N06500,A06500,A85300,MarketHealthIndex,SellForGain,ForeclosureRatio,NegativeEquity,Delinquency,DaysOnMarket,ZHVI
0,1001,490.000000,145.000000,635.000000,88315.000000,1510.000000,89312.833333,64531.000000,486.166667,2512.000000,...,1188.333333,9685.833333,8.500000,2.094542,75.00,5.7575,0.112505,0.072131,77.5,198700.0
1,1002,578.333333,120.000000,681.666667,142458.166667,1588.333333,145486.666667,79139.166667,1103.166667,7696.500000,...,1210.000000,20421.000000,381.500000,3.099581,92.31,NaN,0.060052,0.050725,93.5,315600.0
2,1005,160.000000,36.666667,206.666667,23701.166667,400.000000,23982.166667,18176.666667,89.000000,925.000000,...,315.000000,2418.833333,0.000000,1.054014,50.00,73.3217,0.079545,0.174603,76.0,219500.0
3,1007,555.000000,100.000000,668.333333,94275.500000,1271.666667,95597.166667,71896.666667,450.833333,3163.166667,...,1031.666667,11447.500000,24.833333,2.946270,100.00,NaN,0.057422,0.034014,88.0,260600.0
4,1008,50.000000,6.666667,45.000000,7119.000000,108.333333,7194.500000,5342.166667,22.333333,225.333333,...,88.333333,776.166667,0.000000,1.179644,0.00,NaN,0.071429,0.166667,78.0,224900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13998,99701,373.333333,176.666667,688.333333,74883.500000,1331.666667,75778.333333,52415.500000,291.500000,1718.000000,...,1023.333333,9052.500000,57.833333,9.243381,100.00,NaN,0.173300,0.004785,76.0,184300.0
13999,99705,655.000000,130.000000,993.333333,96718.000000,1471.666667,97462.666667,76748.666667,222.333333,1522.500000,...,1233.333333,10831.000000,8.000000,7.001916,100.00,NaN,0.190909,0.028571,79.0,214500.0
14000,99709,643.333333,158.333333,926.666667,134170.500000,1791.666667,135821.000000,93448.833333,756.000000,3348.166667,...,1500.000000,19324.500000,93.833333,6.566825,92.00,NaN,0.133586,0.003155,74.0,233600.0
14001,99712,321.666667,56.666667,430.000000,57122.333333,721.666667,57677.666667,40777.500000,239.333333,1168.333333,...,608.333333,7773.500000,32.166667,5.771169,90.00,NaN,0.132198,0.030675,76.0,251900.0
